本节的验证码识别是基于tensorflow官方的models中slim中的alexnet，基于slim/nets/alexnet.py文件中的网络结构，来实现验证码识别的多任务学习。

In [ ]:
import os
import tensorflow as tf
from PIL import Image
from nets import nets_factory
import numpy as np

In [ ]:
# 不同字符数量（数字0-9，共10个）
CHAR_SET_LEN = 10
# 图片高度
IMAGE_HEIGHT = 60
# 图片宽度
IMAGE_WIDTH = 160
# 批次
BATCH_SIZE = 25
# TFRecord文件存放的路径
TFRECORD_FILE= '/Users/zoe/Documents/GitHub/July-NLP/TF/DATAGURU/Lec 10/captcha/train.tfrecords'

In [ ]:
x = tf.placeholder(tf.float32, [None, 224, 224])  # 224,224是我们要使用的网络的input的size
y0 = tf.placeholder(tf.float32, [None])
y1 = tf.placeholder(tf.float32, [None])
y2 = tf.placeholder(tf.float32, [None])
y3 = tf.placeholder(tf.float32, [None])

In [ ]:
# 学习率
lr = tf.Variable(0.003, dtype=tf.float32)

In [ ]:
# 从tfrecord中读取数据
def read_and_decode(filename):
    # 根据文件名生成一个队列
    filename_queue = tf.train.string_input_producer([filename])
    reader = tf.TFRecordReader()
    
    # 返回文件名和文件
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'image': tf.FixedLenFeature([], tf.string), 
                                          'label0': tf.FixedLenFeature([], tf.int64),
                                          'label1': tf.FixedLenFeature([], tf.int64),
                                          'label2': tf.FixedLenFeature([], tf.int64),
                                          'label3': tf.FixedLenFeature([], tf.int64),
                                      })
    
    # 获取图片数据
    image = tf.decode_raw(features['image'], tf.uint8)
    # tf.train.shuffle_batch必须确定shape
    image = tf.reshape(image, [224, 224])
    
    # 图片预处理
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, 0.5)
    image = tf.multiply(image, 2.0)
    
    # 获取label
    label0 = tf.cast(features['label0'], tf.int32)
    label1 = tf.cast(features['label1'], tf.int32)
    label2 = tf.cast(features['label2'], tf.int32)
    label3 = tf.cast(features['label3'], tf.int32)
    
    return image, label0, label1, label2, label3

In [ ]:
# 获取图片数据和标签
image, label0, label1, label2, label3 = read_and_decode(TFRECORD_FILE)

In [ ]:
image

In [ ]:
with tf.Session() as sess:
    print(sess.run(label0))